In [1]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms, models
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import OrderedDict
import PIL 
from PIL import Image

In [2]:
def load_checkpoint(filepath):
        checkpoint = torch.load(filepath)
        ###
        model = get_model(architecture=checkpoint['artitecture'], hidden=checkpoint['hidden'])
        #model = get_model(architecture='vgg13', hidden = 100)
        ####
        # epochs = checkpoint['epochs']
        model.load_state_dict(checkpoint['state_dict'])
        model.class_to_idx = checkpoint['image_datasets']
        #optimizer.load_state_dict(checkpoint['optimizer'])
        model.class_to_idx = {model.class_to_idx[k]: k for k in model.class_to_idx}
        
        #model.artitecture='vgg13'
        #model.hidden=100
        #model = get_model(artitecture=model.artitecture, hidden=model.hedden)
        return model

In [3]:
def process_image(image):
    
    img_loader = transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(224), 
        transforms.ToTensor()])
    
    pil_image = Image.open(image)
    pil_image = img_loader(pil_image).float()
    
    np_image = np.array(pil_image)    
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np.transpose(np_image, (1, 2, 0)) - mean)/std    
    np_image = np.transpose(np_image, (2, 0, 1))
            
    return np_image

### Define del model

In [4]:
def get_model(architecture, hidden):
    from collections import OrderedDict
    #hidden=[1024, 128, 102]
    if architecture == 'vgg13':
        model = models.vgg16(pretrained=True)
        classifier_input_size = model.classifier[0].in_features
    elif architecture == 'densenet121':
        model = models.densenet121(pretrained=True)
        classifier_input_size = 1024 
    else:
        print("error")
        stop()
    hidden = hidden   
    classifier_output_size = 102
    # Classificator
    
    #model = models.vgg16(pretrained=True)
    #classifier_input_size = model.classifier[0].in_features

    classifier = nn.Sequential(OrderedDict([           
                              ('fc1', nn.Linear(classifier_input_size, hidden)), 
                              ('relu1', nn.ReLU()),
                              ('drop',nn.Dropout(p=0.2)),
                              ('fc2', nn.Linear(hidden,classifier_output_size)),
                              ('output', nn.LogSoftmax(dim=1))
                              ]))

    model.classifier = classifier

    model.cuda()
    
    return model


### Predict function

In [5]:
def predict(image_path, model,topk=5):

    gpu_check = torch.cuda.is_available()
    model.eval()
    np_array =process_image(image_path)
    tensor = torch.from_numpy(np_array)
    var_inputs = Variable(tensor.float().cuda(), volatile=True)
    output = model.forward(var_inputs.unsqueeze(0))
    #probs, classes
    ps= torch.exp(output).data.topk(topk)
    probs = ps[0].cpu() if gpu_check else ps[0]
    classes = ps[1].cpu() if gpu_check else ps[1]
   
    classes_ = list()
    for item in classes.numpy()[0]:
        classes_.append(model.class_to_idx[item])
    #return probs, classes
    return probs.numpy()[0], classes_

### Testing Main

In [6]:
NOOOOOOOOOOO
image_path = test_dir + '/54/image_05413.jpg'
#model=get_model()
#model.cuda()
model = load_checkpoint('test.pth')
#model.class_to_idx = {model.class_to_idx[k]: k for k in model.class_to_idx}
probs, classes = predict(image_path, model)

print(probs)
print(classes)

NameError: name 'NOOOOOOOOOOO' is not defined

In [7]:
def mainfunction(checkpoint, image_path, map_file='cat_to_name.json', topk=5):
        print("Start")
        model = load_checkpoint(filepath=checkpoint)
        print("phase1")
        probs, classes = predict(image_path, model,topk=topk)
        print("phase2")
        
        with open(map_file, 'r') as f:
            cat_to_name = json.load(f)
        print("Flower Name: Associated Probability: ")
        for i in range(0, len(classes)):
            print("{} {:.3f}".format(cat_to_name[classes[i]], probs[i]))
        
        return probs, classes
        

In [8]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
probs, classes = mainfunction(checkpoint='test.pth', image_path = test_dir + '/54/image_05413.jpg', topk=5)

Start


/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


phase1
phase2
Flower Name: Associated Probability: 
passion flower 0.085
pink-yellow dahlia 0.069
barbeton daisy 0.062
common dandelion 0.055
colt's foot 0.055


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys


In [ ]:
print(probs)
print(classes)